In [41]:
import numpy as np
from scipy import sparse

# 行列形式のリンク情報をベクトル形式に変換する関数
def trans_linkMat_to_linkVec(linkMat, init_incMat, term_incMat):

    temp_linkMat = init_incMat.T @ linkMat @ term_incMat
    linkVec = np.diag(temp_linkMat.toarray())

    return linkVec

# ベクトル形式のリンク情報を行列形式に変換する関数
def trans_linkVec_to_linkMat(linkVec, init_incMat, term_incMat):

    num_vec = linkVec.shape[0]
    row = np.arange(num_vec)
    col = np.arange(num_vec)

    temp_linkMat = sparse.csr_matrix((linkVec, (row, col)))
    linkMat = init_incMat @ temp_linkMat @ term_incMat.T

    return linkMat

# 重み行列（exp(-theta*link_cost)）作成
def make_link_weight(cost_vec, theta):

    link_weight = np.exp(-theta * cost_vec)

    return link_weight

# 期待最小費用行列を作成する関数(起点×目的のノード)
def calc_expected_minCost_mat(weight_mat, conv_judge):

    num_vec = weight_mat.shape[0]
    data = np.ones(num_vec)
    row = np.arange(num_vec)
    col = np.arange(num_vec)

    exp_minCost = sparse.csr_matrix((data, (row, col)))
    temp_exp_minCost = exp_minCost.copy()

    while np.max(temp_exp_minCost) > conv_judge:
        temp_exp_minCost = temp_exp_minCost @ weight_mat
        exp_minCost += temp_exp_minCost

    return exp_minCost

# 期待最小費用からリンクの条件付き選択確率を計算する関数
def calc_choPer(weight_mat, exp_minCost, orig_node_id):

    temp_minCost_vec = exp_minCost[orig_node_id, :]
    data = temp_minCost_vec.data
    row = temp_minCost_vec.indices
    col = temp_minCost_vec.indices

    temp_mat = sparse.csr_matrix((data, (row, col)), shape=weight_mat.shape)
    per_nume = temp_mat @ weight_mat

    # ここはもう少し効率化できそう
    per_mat = np.divide(per_nume.toarray(), temp_minCost_vec.toarray(), out=np.zeros_like(per_nume.toarray()), where=temp_minCost_vec.toarray() != 0)
    per_mat = sparse.csr_matrix(per_mat)

    return per_mat

# ノードフローを計算する関数(demandをスパース行列にすると早そう)
def calc_nodeFlow(per_mat, demand):

    num_vec = per_mat.shape[0]
    data = np.ones(num_vec)
    row_col = np.arange(num_vec)

    node_per_mat = sparse.csr_matrix((data, (row_col, row_col)))
    temp_per_mat = sparse.csr_matrix((data, (row_col, row_col)))

    while np.max(temp_per_mat) > 0.0:
        temp_per_mat = temp_per_mat @ per_mat
        node_per_mat += temp_per_mat

    nodeFlow = (node_per_mat @ demand.T).T

    return nodeFlow

# リンクフローを計算する関数
def calc_linkFlow(per_mat, nodeFlow):
    
    # ここはもう少し速くできそう
    linkFlow = np.multiply(nodeFlow.toarray(), per_mat.toarray())
    linkFlow = sparse.csr_matrix(linkFlow)

    return linkFlow

# ロジット配分を計算する関数
def LOGIT(cost_vec, tripsMat, init_incMat, term_incMat, theta, conv_judge=0.0):

    link_weight = make_link_weight(cost_vec, theta)
    weight_mat = trans_linkVec_to_linkMat(link_weight, init_incMat, term_incMat)
    exp_minCost = calc_expected_minCost_mat(weight_mat, conv_judge)

    total_link_flow = sparse.csr_matrix(([], ([], [])), shape=(init_incMat.shape[0], init_incMat.shape[0]))

    for orig_node_id in range(tripsMat.shape[0]):

        per_mat = calc_choPer(weight_mat, exp_minCost, orig_node_id)
        node_flow = calc_nodeFlow(per_mat, tripsMat[orig_node_id])
        link_flow = calc_linkFlow(per_mat, node_flow)
        total_link_flow += link_flow

    link_flow_vec = trans_linkMat_to_linkVec(total_link_flow, init_incMat, term_incMat)

    return link_flow_vec


In [42]:
import time
import pandas as pd
import openmatrix as omx

# !!=============================================================!!
#   データの読み取り関数
# !!=============================================================!!

# function to read <NUMBER OF ZONES>
def read_num_zones(netfile):

    file_data = open(netfile)
    num_zones = 0

    for i in range(8):
        line = file_data.readline()
        if "<NUMBER OF ZONES>" in line:
            num_zones = int(line.split('\t')[0][17:])
            # print(num_zones)

    return num_zones

# function to read <NUMBER OF NODES>
def read_num_nodes(netfile):

    file_data = open(netfile)
    num_nodes = 0

    for i in range(8):
        line = file_data.readline()
        if "<NUMBER OF NODES>" in line:
            num_nodes = int(line.split('\t')[0][17:])
            # print(num_nodes)

    return num_nodes

# ネットワークデータを読み取る関数
def read_net(netfile):

    net = pd.read_csv(netfile, skiprows=8, sep='\t')
    trimmed = [s.strip().lower() for s in net.columns]
    net.columns = trimmed

    # And drop the silly first andlast columns
    net.drop(['~', ';'], axis=1, inplace=True)

    return net

# OD需要を読み取る関数
def read_trips(tripsfile):

    f = open(tripsfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig = int(orig[0])

        d = [eval('{'+a.replace(';', ',').replace(' ', '') + '}')
             for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            # We map values to a index i-1, as Numpy is base 0
            mat[i, j] = matrix.get(i+1, {}).get(j+1, 0)

    index = np.arange(zones) + 1

    myfile = omx.open_file('demand.omx', 'w')
    myfile['matrix'] = mat
    myfile.create_mapping('taz', index)
    myfile.close()

    return matrix
       
# !!=============================================================!!
#   データ形式の変換関数 (引数として使える形に)
# !!=============================================================!!

# tripsデータ（OD需要）を行列（起点ノード×全ノード）形式に変換する関数
def make_tripsMat(trips, num_zones, num_nodes):
    tripsMat = np.zeros((num_zones, num_nodes))
    for orig_node in trips.keys():
        for dest_node in trips[orig_node].keys():
            tripsMat[orig_node-1, dest_node-1] = trips[orig_node][dest_node]
    tripsMat = sparse.csr_matrix(tripsMat)
    return tripsMat

# 起点ノード×リンクの接続行列を作成する関数
def make_init_incMat(links, num_nodes):
    init_incMat = np.zeros((num_nodes, len(links)), dtype=int)
    for index, link in links.iterrows():
        init_incMat[int(link['init_node'])-1, index] = 1 
    init_incMat = sparse.csr_matrix(init_incMat)
    return init_incMat

# 終点ノード×リンクの接続行列を作成する関数
def make_term_incMat(links, num_nodes):
    term_incMat = np.zeros((num_nodes, len(links)), dtype=int)
    for index, link in links.iterrows():
        term_incMat[int(link['term_node'])-1, index] = 1

    term_incMat = sparse.csr_matrix(term_incMat)
    return term_incMat

# !!=============================================================!! 
#   LOGIT配分の実行
# !!=============================================================!!

# データの読み取り
links = read_net('./SiouxFalls_net.tntp')
trips = read_trips('./SiouxFalls_trips.tntp')
num_zones = read_num_zones('./SiouxFalls_net.tntp')
num_nodes= read_num_nodes('./SiouxFalls_net.tntp')
# print(links)

# リンクの接続情報を行列形式に変換
init_incMat = make_init_incMat(links, num_nodes)
term_incMat = make_term_incMat(links, num_nodes)
# リンクコストをベクトル形式に変換
costVec = np.array(links['free_flow_time'])
# tripsを行列形式に変換
tripsMat = make_tripsMat(trips, num_zones, num_nodes)

# LOGIT配分を計算
start_time = time.process_time()
LOGIT_flow = LOGIT(costVec, tripsMat, init_incMat, term_incMat, theta=1.0, conv_judge=0.0)
end_time = time.process_time()
print('LOGIT_flow = ', LOGIT_flow)
print('LOGIT_time = ', end_time - start_time)

# pandas形式にしたいなら……
# links['LOGIT_FLOW'] = LOGIT_flow
# print('Links = ', links) 
                                    

LOGIT_flow =  [ 3644.30763886  6268.66147458  3644.3076979   6381.01928116
  6268.66141554  9008.87427442  8173.92719439  9008.84996707
 14361.98608008  6955.37198879 14362.107923   10240.23812428
  7771.08558638  6381.0193402  10240.34494612 17313.53317034
  8646.85724327 13871.30810066 17313.64005121  8646.6610578
   758.15450782 14102.98543314  7771.10060747   758.15247523
 17134.13945043 17234.15243892 21548.13794086 14387.10410583
 27041.23241263  1173.36075149  7055.22583852 21348.18588348
 11462.20129985 14835.52861059  8173.9514427  11461.60872971
 12762.73146146 12862.16313963 12132.44476692 14836.02297308
  8785.54060993  8772.34152233 14486.31939792  8785.95597581
 18042.60652303 23535.09766945 14102.89816113 27141.6268414
 28042.53197111 13666.83226353  1173.7160765  28042.94362084
 23650.56070985 13871.50428614 13766.72777057 10909.61496082
 18042.37843892 23651.32768459  6960.85503312 10909.70665333
  6961.39392375  4096.22475407  7510.56149398  4030.13615178
 15312.97153